In [ ]:
with open('input.txt') as file:
    data = file.read()

data = [int(num) for num in data.split(' ')]

In [ ]:
# Silver
import math
max_iter = 25
current_order = data

def split_number(x,digits):
    left = x // 10 ** (digits)
    right = x % (10 ** digits)
    return left, right

for iteration in range(max_iter):
    new_order = []
    for stone in current_order: 

        if stone == 0:
            new_order.append(1)
            continue
        digits = int(math.log10(stone)) + 1
        if digits % 2 == 0:
            stone_digits = str(stone)
            new_order.extend(split_number(stone,digits//2))
            continue
        new_order.append(stone * 2024)
    current_order = new_order    

print(len(current_order))

In [ ]:
# Gold, bechmark 1000 iterations 10 sec
import math
from collections import Counter
max_iter = 75
ltable = []
logic = {}

def split_number(x,digits):
    left = x // 10 ** (digits)
    right = x % (10 ** digits)
    return left, right

def process_stones(stones, new_order):
    indexes = []
    for stone in stones:
        if stone in ltable:
            indexes.append(ltable.index(stone))
        else:
            ltable.append(stone)
            indexes.append(len(ltable) - 1)
    new_order.extend(indexes)
    logic[stone_id] = indexes


current_order = []
for ii,num in enumerate(data):
    current_order.append(ii)
    ltable.append(num)


precalculated = [] 

for iteration in range(max_iter):
    new_order = []
    for stone_id in current_order: 
        
        # If previously encountered value is found add it to the list and continue
        if stone_id in logic.keys():
            # Save (iteration, stone_id) pairs for future processing
            precalculated.append((iteration, stone_id))
            continue

        if ltable[stone_id] == 0:
            next_stone = 1
            process_stones([next_stone], new_order)
            continue

        digits = int(math.log10(ltable[stone_id])) + 1
        if digits % 2 == 0:
            next_stones = split_number(ltable[stone_id], digits//2)
            process_stones(next_stones, new_order)
            continue

        next_stone = ltable[stone_id] * 2024
        process_stones([next_stone],new_order)

    current_order = new_order    



# Wacky shit, I cant be bothered to explain
# Basic idea is to utilise counter and dynamically calculate thing along the way
counter = Counter(precalculated)
current_iter = 1

while current_iter < max_iter:
    results = sorted(counter.keys())
    iteration = 0
    ii = 0
    while iteration <= current_iter and ii < len(results):
        iteration, stone_id = results[ii]
        # Since it is sorted list we can just break the loop when we reach next iter
        if iteration > current_iter:
            break
        next_stones = logic[stone_id]
        for stone in next_stones:
            # Counters are nice, when empty you can add one and the key magically appears
            counter[(iteration+1, stone)] += counter[results[ii]]
        # Remove old key and count    
        counter.pop(results[ii])
        ii += 1
    current_iter += 1
    
# The current_order is actually empty from iteration 50+ and all elements can be calculated through this counter implementations which is probably something like O(n log(n))
len(current_order) + counter.total()
